In [ ]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
import cv2
from skimage import color
from skimage import io
from skimage.filters import gaussian
from scipy.ndimage import binary_fill_holes

In [ ]:
def smooth_image(gray):
    # threshold
    thresh = cv2.threshold(gray, 32, 255, cv2.THRESH_BINARY)[1]
    # blur threshold image
    blur = cv2.GaussianBlur(thresh, (0,0), sigmaX=3, sigmaY=3, borderType = cv2.BORDER_DEFAULT)
    # stretch so that 255 -> 255 and 127.5 -> 0
    stretch = rescale_intensity(blur, in_range=(127.5,255), out_range=(0,255)).astype(np.uint8)
    # threshold again
    thresh2 = cv2.threshold(stretch, 0, 255, cv2.THRESH_BINARY)[1]
    # get external contour
    contours = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    big_contour = max(contours, key=cv2.contourArea)
    # draw white filled contour on black background as mas
    contour = np.zeros_like(gray)
    cv2.drawContours(contour, [big_contour], 0, 255, -1)
    # dilate mask for dark border
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (12,12))
    dilate = cv2.morphologyEx(contour, cv2.MORPH_CLOSE, kernel)
    # apply morphology erode
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))
    dilate = cv2.morphologyEx(dilate, cv2.MORPH_ERODE, kernel)
    # blur dilate image
    blur2 = cv2.GaussianBlur(dilate, (3,3), sigmaX=0, sigmaY=0, borderType = cv2.BORDER_DEFAULT)
    # stretch so that 255 -> 255 and 127.5 -> 0
    mask = rescale_intensity(blur2, in_range=(127.5,255), out_range=(0,255))
    #return cv2.bitwise_and(gray, gray, mask=mask.astype(np.uint8))
    return cv2.bitwise_and(gray, mask.astype(np.uint8), mask=None)

def find_contours(image):
    # red color boundaries [B, G, R]
    lower = 0
    upper = 1000

    # create NumPy arrays from the boundaries
    #lower = np.array(lower, dtype="uint8")
    #upper = np.array(upper, dtype="uint8")

    # find the colors within the specified boundaries and apply
    # the mask
    
    mask = cv2.inRange(image, lower, upper)
    output = cv2.bitwise_and(image, image, mask=mask)

    ret,thresh = cv2.threshold(mask, 40, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print(f'len contour={len(contours)})
    if len(contours) != 0:
        
        # draw in blue the contours that were founded
        cv2.drawContours(output, contours, -1, 255, 3)

        # find the biggest countour (c) by the area
        c = max(contours, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)

        # draw the biggest contour (c) in green
        cv2.rectangle(output,(x,y),(x+w,y+h),(0,255,0),2)
        
    return output


In [ ]:
image = img.copy()
ret,thresh = cv2.threshold(image, 0, 200, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
areaArray = []
for contour in contours:
    area = cv2.contourArea(contour)
    print(area)
    areaArray.append(area)
# first sort the array by area
sorteddata = sorted(zip(areaArray, contours), key=lambda x: x[0], reverse=True)
largest_contour = sorteddata[0][1]
output = cv2.fillPoly(image, pts =[largest_contour], color=255)
plt.imshow(output, cmap="gray")
print(output.shape)

In [ ]:
animal = 'DK37'
DIR = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'
INPUT = os.path.join(DIR, 'C1', 'thumbnail')
file = '000.tif'
infile = os.path.join(INPUT, file)

In [ ]:
img = io.imread(infile)
img = np.rot90(img, 3, axes=(1,0))
img = (img/256).astype(np.uint8)
print('img',img.dtype, img.shape)
plt.imshow(img)

In [ ]:
lowerbound = 0
upperbound = 255
#all pixels value above lowerbound will  be set to upperbound 
_, thresh = cv2.threshold(img.copy(), lowerbound, upperbound, cv2.THRESH_BINARY_INV)
kernel_size = (2,2)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
thresh = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel)
#smoothed = gaussian(thresh, 50)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
smoothed = cv2.morphologyEx(thresh, cv2.MORPH_ERODE, kernel)
inverted_thresh = cv2.bitwise_not(smoothed)
filled_thresh = binary_fill_holes(inverted_thresh).astype(np.uint8)
cleaned = cv2.bitwise_and(img,img, mask=filled_thresh)

In [ ]:
contours = find_contours(img)
print('contours', contours.dtype, contours.shape)

In [ ]:
images = [contours, cleaned, smoothed, thresh, filled_thresh, img, inverted_thresh]
titles = ['contours', 'cleaned', 'smoothed', 'thresh', 'filled_thresh', 'img', 'inverted_thresh']

_, axs = plt.subplots(len(images),1, figsize=(12, 12))
axs = axs.flatten()
for image, ax, title in zip(images, axs, titles):
    ax.set_title(f'{title}')
    ax.imshow(image, cmap="gray")
    plt.tight_layout()
#plt.show()